In [1]:
import numpy as np
import caffe
import errno
import glob
import os
os.environ['LMDB_FORCE_CFFI'] = '0'


ImportError: libcudart.so.8.0: cannot open shared object file: No such file or directory

In [ ]:
import lmdb

In [ ]:
# Paths error preventing functions

def make_sure_path_exists(directory):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)
        except OSError as error:
            if error.errno != errno.EEXIST:
                raise
                
def make_sure_file_exists(infile):
    if not os.path.isfile(infile):
        try:
            with open(infile, 'w') as f:
                print 'file found...'
        except OSError as error:
            if error.errno != errno.EEXIST:
                raise

In [ ]:
#Directories definition function

#Directory to build processed dataset
work_path = "/home/gonzalo/dev/post_data/MIT" #SCfaces"
print work_path #for debug purposes
make_sure_path_exists(work_path)

#Directory to from where to extract data
root_dir = "/home/gonzalo"
media_dir = "/media/gonzalo/OS"

#Set manually for output filename 
mode = 'train'
step = 'step_0'

make_sure_path_exists(root_dir)
data_path = os.path.join(media_dir,"dev", "facerec", "MIT_frontalized_detected", mode, step)#"training-synthetic") #,'SCface_database','mugshot_rotation_all') #
print (data_path) #for debug purposes

# List of subjects numbers for folder creation
name =["%04d" % x for x in range(10)] #for MIT-CBCL

# Let's pretend this is interesting data
X_list = []
y_list = []

In [ ]:
def front_database_build(name, data_path, work_path, mode):
    ''' 
    Given the name of the subject in the variable -name-, the path from where to obtain the images, 
    the path to the destiny folder in which to save the database and the eyemask parameter used for
    frontalization of the faces, this routine takes the images from data_path and saves the 
    frontalized face obtained in the work_path separating pictures in folders for each individual. 
    '''
    for n in name[:]:
        pathdir = os.path.join(work_path, mode, n)
    
        print pathdir #for debug purposes
        make_sure_path_exists(pathdir)
        search_filename = n +  "*.jpg" #
        search_location = os.path.join(data_path,search_filename)
        print search_location #for debug purposes
    
        for filepath in glob.glob(search_location):
            
            # Extract filename from the filepath found at search location
            tail = re.split(r"[.]", os.path.basename(filepath))
            tail = str(tail[0])
            print tail #for debug purposes
            out_name = tail + '.jpg'
            
            # For MIT-CBCL dataset file naming, break name into the image data provided in filename
            # 'subject' - 'face rotation' - etc
            file_data = re.split(r"[_]", tail)
            #print file_data #for debug purposes
            
            #Copy only the images that have less than face_rot rotation angle
            if os.path.isfile(filepath): #and face_rot<float(file_data[1]):
    
                image = cv2.imread(filepath,1)# cv2.IMREAD_GRAYSCALE)

                if image is not None:
                    X_list.append(image)
                    y_list.append(n)
    X = np.array(X_list)
    y = np.array(y_list)
    
    return X_np, y_np

In [ ]:
# We need to prepare the database for the size. We'll set it 10 times
# greater than what we theoretically need. 
map_size = X.nbytes * 10

env = lmdb.open('mylmdb', map_size=map_size)

with env.begin(write=True) as txn:
    # txn is a Transaction object
    for i in range(N):
        datum = caffe.proto.caffe_pb2.Datum()
        datum.channels = X.shape[1]
        datum.height = X.shape[2]
        datum.width = X.shape[3]
        datum.data = X[i].tobytes()  # or .tostring() if numpy < 1.9
        datum.label = int(y[i])
        str_id = '{:08}'.format(i)

        # The encode is only essential in Python 3
        txn.put(str_id.encode('ascii'), datum.SerializeToString())